INSTALL PACKAGES

In [ ]:
%pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


LOAD LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import os
import tqdm.notebook as tqdm

In [ ]:
os.chdir('/content/drive/MyDrive/data')

In [ ]:
df_test1 = pd.read_csv('test_data.csv')
df_test2 = pd.read_csv('test_data2.csv')
# concatenate the two dataframes
df_test = pd.concat([df_test1, df_test2], ignore_index = True)
# drop Unnamed: 0 column from df_test
df_test = df_test.drop(['Unnamed: 0'], axis = 1)
df_test

,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,country,lang,replyCount
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,nikoonya,2011-01-10 16:24:18+00:00,False,339.0,214.0,0.0,1.0,"Kansas City, MO",🍷 Co-founder & Doc’s List @UvaVault | Holder o...,United States,en,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3802.0,4933.0,0.0,0.0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...,United States,en,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = pd.read_csv('ChatGPT.csv')

# create a new column for the tweetID
df['tweetID'] = np.nan

# tweetID is the numbers after the last slash in the url
for i in range(len(df)):
    df['tweetID'][i] = df['Url'][i].split('/')[-1]

<ipython-input-14-8daf9a2d318e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweetID'][i] = df['Url'][i].split('/')[-1]


In [ ]:
# remove rows after 154084
df_og = df[:154084]

# drop tweetID and Url columns
df_og = df_og.drop(['tweetID', 'Url'], axis = 1)

In [ ]:
# get 200 rows from df_test that are not null
non_null_df_test = df_test[df_test.tweet.notnull()]
non_null_df_test = non_null_df_test.tail(200)
non_null_df_test['survive'] = 1
non_null_df_test.drop(columns=['country', 'lang', 'replyCount'], inplace=True)

# get 200 rows from df_test that are null
null_df_test = df_test[df_test.tweet.isnull()]
null_df_test = null_df_test.tail(200)
null_df_test_updated = df_og.iloc[null_df_test.index]
null_df_test_updated['survive'] = 0
null_df_test_updated.columns = list(non_null_df_test.columns)

# concatenate the two dataframes
test1 = pd.concat([non_null_df_test, null_df_test_updated], ignore_index = True)

<ipython-input-16-c6a6fb2cf302>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_df_test_updated['survive'] = 0


In [ ]:
# get rows that are not null from df_test
df2 = df_test[df_test.tweet.notnull()]

df2 = df2.head(500)

# get rows that are null from df_test
df3 = df_test[df_test.tweet.isnull()]

df3 = df3.head(500)

In [ ]:
# get index numbers from df3
df3_indx = df3.index

# get rows from df_og with the index numbers from df3_indx
df3_og = df_og.iloc[df3_indx]

In [ ]:
df2.drop(columns=['country', 'lang', 'replyCount'], inplace=True)
df2['survived'] = 1
df3_og['survived'] = 0

<ipython-input-19-b23f3f409406>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_og['survived'] = 0


In [ ]:
df3_og.columns = list(df2.columns)

In [ ]:
train_df = pd.concat([df2, df3_og], ignore_index = True)
train_df

,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,survived
0,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,nikoonya,2011-01-10 16:24:18+00:00,False,339.0,214.0,0.0,1.0,"Kansas City, MO",🍷 Co-founder & Doc’s List @UvaVault | Holder o...,1
1,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3802.0,4933.0,0.0,0.0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...,1
2,2023-01-29 14:39:11+00:00,"Excellent piece.""Much of what is taught in #ST...",didem_un_ates,2009-09-01 14:19:17+00:00,False,1699.0,1433.0,4.0,9.0,"London, England",@Microsoft #Data #AI #Metaverse l #Responsible...,1
3,2023-01-29 14:35:59+00:00,"#ChatGPT gets it… solve cons from list A, with...",UvaVault,2022-07-22 00:49:21+00:00,False,343.0,246.0,0.0,6.0,"Atlanta, GA",750ml Sold Out. Asset-backed NFT🍷+ Cellaring +...,1
4,2023-01-29 14:30:14+00:00,“If you’re doing original reporting and uneart...,Joab_Jackson,2009-03-07 15:56:20+00:00,False,7110.0,6719.0,0.0,0.0,Bed-Stuy NYC / Sleepy Creek WV,Listening to the Technology / Editor-in-Chief ...,1
...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-01-29 11:23:57+00:00,OpenAI's #ChatGPT is writing a new book.\n\n👇 ...,TheEternalStory,2023-01-18 22:19:18+00:00,False,0.0,0.0,0.0,0.0,NaN,Asla bitmeyen bir kitap.,0
996,2023-01-29 11:23:56+00:00,I am getting more impressed by #ChatGPT’s abil...,weldeiry,2008-04-27 23:44:29+00:00,False,10651.0,9241.0,1.0,4.0,Pennsylvania,Physician-Scientist; Med-Onc; Pubs https://t.c...,0
997,2023-01-29 11:22:52+00:00,#ChatGPT is better than human authors.\n\n👇 Click,DexAuctor,2023-01-20 06:55:44+00:00,False,1.0,0.0,0.0,1.0,NaN,"Co-writer of the Twitter-based book ""The Etern...",0
998,2023-01-29 11:22:41+00:00,Google announced a new ai chatGPT application....,NareshAskani,2017-02-28 08:27:59+00:00,False,15.0,51.0,0.0,2.0,"Hyderabad, India","Son of agriculture man, Thinker, Developer, D...",0


In [ ]:
### https://github.com/mehranshakarami/AI_Spectrum/blob/main/2022/Sentiment_Analysis/tw-sentiment.py#L38 ### BY MEHRAN SHAKARAMI

def tweet_analysis(tweet):
    # precprcess tweet
    tweet_words = []

    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)

    # load model and tokenizer
    roberta = "cardiffnlp/twitter-roberta-base-sentiment"

    model = AutoModelForSequenceClassification.from_pretrained(roberta)
    tokenizer = AutoTokenizer.from_pretrained(roberta)

    labels = ['Negative', 'Neutral', 'Positive']

    # sentiment analysis
    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
    # output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
    output = model(**encoded_tweet)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    return scores
    # for i in range(len(scores)):
        
    #     l = labels[i]
    #     s = scores[i]
    #     print(s)
        

In [ ]:
analysis_df = pd.DataFrame(columns= ['tweet', 'positive_prob', 'neutral_prob', 'negative_prob'])


for i, id in tqdm.tqdm(enumerate(train_df.tweet), total= len(train_df.tweet)):
    analysis_df = analysis_df.append({'tweet': str(id), 'positive_prob': tweet_analysis(id)[0], 
                                      'neutral_prob': tweet_analysis(id)[1], 'negative_prob': tweet_analysis(id)[2]}, ignore_index = True)

analysis_df

  0%|          | 0/1000 [00:00<?, ?it/s]

<ipython-input-129-773a15217f88>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  analysis_df = analysis_df.append({'tweet': str(id), 'positive_prob': tweet_analysis(id)[0],
<ipython-input-129-773a15217f88>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  analysis_df = analysis_df.append({'tweet': str(id), 'positive_prob': tweet_analysis(id)[0],
<ipython-input-129-773a15217f88>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  analysis_df = analysis_df.append({'tweet': str(id), 'positive_prob': tweet_analysis(id)[0],
<ipython-input-129-773a15217f88>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  analysis_df = analysis_df.append({'tweet

,tweet,positive_prob,neutral_prob,negative_prob
0,#ChatGPT thank u for my pitch deck bullets.,0.004381,0.154541,0.841077
1,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,0.024142,0.435439,0.540419
2,"Excellent piece.""Much of what is taught in #ST...",0.008182,0.176340,0.815478
3,"#ChatGPT gets it… solve cons from list A, with...",0.030797,0.497710,0.471493
4,“If you’re doing original reporting and uneart...,0.143114,0.592953,0.263933
...,...,...,...,...
995,OpenAI's #ChatGPT is writing a new book.\n\n👇 ...,0.006266,0.547368,0.446367
996,I am getting more impressed by #ChatGPT’s abil...,0.002790,0.048222,0.948988
997,#ChatGPT is better than human authors.\n\n👇 Click,0.039165,0.303260,0.657574
998,Google announced a new ai chatGPT application....,0.001416,0.019285,0.979299


In [ ]:
analysis_df.to_csv('Sentiment_data.csv')

In [ ]:
# concatenate the two test dataframes
test1 = pd.concat([non_null_df_test, null_df_test_updated], ignore_index = True)
test1

,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,survive
0,2022-12-08 22:57:31+00:00,"Have already used #ChatGPT to evaluate #code,...",roblew9013,2009-07-15 02:47:30+00:00,False,211.0,753.0,0.0,1.0,"San Francisco, CA",Sharing thoughts on: Life Science 🧬 Data Scien...,1
1,2022-12-08 22:51:07+00:00,Everyone who works in tech is about to have a ...,cantblockjo,2018-08-10 00:11:22+00:00,False,15.0,103.0,0.0,0.0,"London, England",Some things are better left unsaid.,1
2,2022-12-08 21:51:50+00:00,Just had #chatGPT churn out 12 blog posts for ...,roblew9013,2009-07-15 02:47:30+00:00,False,211.0,753.0,0.0,0.0,"San Francisco, CA",Sharing thoughts on: Life Science 🧬 Data Scien...,1
3,2022-12-08 21:32:55+00:00,Amazing endless uses for #ChatGPT !\n👇🏼,docmaarten,2009-07-02 21:53:21+00:00,False,485.0,489.0,0.0,0.0,Adelaide Australia,skill acquisition • motor learning • cognitive...,1
4,2022-12-08 21:15:03+00:00,#ChatGPT is doing all of my email replies from...,liamchampton,2011-07-04 21:21:28+00:00,False,800.0,517.0,0.0,4.0,"London, UK",Sr. Regional Cloud Advocate @Microsoft & @Auth...,1
...,...,...,...,...,...,...,...,...,...,...,...,...
395,2022-12-07 03:30:08+00:00,My first questions … #ChatGPT https://t.co/Zze...,anandavardhanan,2014-03-30 03:07:58+00:00,False,1732.0,126.0,0.0,0.0,Mumbai,"Links, likes, and RT's need not be endorsement...",0
396,2022-12-07 03:29:59+00:00,@anobelodisho @CanesDavid @wandering_gu @urete...,TylerSbrt,2019-11-13 06:08:36+00:00,False,1634.0,513.0,0.0,0.0,"San Diego, CA",#radonc and #cancer scientist @UCSanDiego. Qua...,0
397,2022-12-07 03:29:29+00:00,One example of where the AI train could be hea...,davidbutlermd,2013-04-04 05:42:19+00:00,False,1744.0,1932.0,1.0,1.0,"Sacramento, CA",✾ A 'ridiculous optimist' about fixing US #hea...,0
398,2022-12-07 03:29:23+00:00,I broke Chat GPT #OpenAIChat #ChatGPT https://...,whoisorin,2009-03-12 07:30:52+00:00,False,896.0,829.0,0.0,0.0,Chandler AZ,Founder https://t.co/8ZiOaU5vpz Inefficiency b...,0


In [ ]:
analysis_test_df = pd.DataFrame(columns= ['tweet', 'positive_prob', 'neutral_prob', 'negative_prob'])


for i, id in tqdm.tqdm(enumerate(test1.tweet), total= len(test1.tweet)):
    analysis_test_df = analysis_test_df.append({'tweet': str(id), 'positive_prob': tweet_analysis(id)[0], 
                                      'neutral_prob': tweet_analysis(id)[1], 'negative_prob': tweet_analysis(id)[2]}, ignore_index = True)

analysis_test_df

  0%|          | 0/400 [00:00<?, ?it/s]

<ipython-input-24-d309c0551623>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  analysis_test_df = analysis_test_df.append({'tweet': str(id), 'positive_prob': tweet_analysis(id)[0],
<ipython-input-24-d309c0551623>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  analysis_test_df = analysis_test_df.append({'tweet': str(id), 'positive_prob': tweet_analysis(id)[0],
<ipython-input-24-d309c0551623>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  analysis_test_df = analysis_test_df.append({'tweet': str(id), 'positive_prob': tweet_analysis(id)[0],
<ipython-input-24-d309c0551623>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  analysis_test_

,tweet,positive_prob,neutral_prob,negative_prob
0,"Have already used #ChatGPT to evaluate #code,...",0.003509,0.102342,0.894149
1,Everyone who works in tech is about to have a ...,0.001352,0.028491,0.970158
2,Just had #chatGPT churn out 12 blog posts for ...,0.340069,0.532387,0.127544
3,Amazing endless uses for #ChatGPT !\n👇🏼,0.002010,0.035230,0.962759
4,#ChatGPT is doing all of my email replies from...,0.003360,0.193824,0.802817
...,...,...,...,...
395,My first questions … #ChatGPT https://t.co/Zze...,0.044982,0.890369,0.064649
396,@anobelodisho @CanesDavid @wandering_gu @urete...,0.266457,0.700630,0.032913
397,One example of where the AI train could be hea...,0.001767,0.095859,0.902373
398,I broke Chat GPT #OpenAIChat #ChatGPT https://...,0.208422,0.690335,0.101243


In [ ]:
analysis_test_df.to_csv('Sentiment_test_data.csv')

In [ ]:
analysis_test_df

,tweet,positive_prob,neutral_prob,negative_prob
0,"Have already used #ChatGPT to evaluate #code,...",0.003509,0.102342,0.894149
1,Everyone who works in tech is about to have a ...,0.001352,0.028491,0.970158
2,Just had #chatGPT churn out 12 blog posts for ...,0.340069,0.532387,0.127544
3,Amazing endless uses for #ChatGPT !\n👇🏼,0.002010,0.035230,0.962759
4,#ChatGPT is doing all of my email replies from...,0.003360,0.193824,0.802817
...,...,...,...,...
395,My first questions … #ChatGPT https://t.co/Zze...,0.044982,0.890369,0.064649
396,@anobelodisho @CanesDavid @wandering_gu @urete...,0.266457,0.700630,0.032913
397,One example of where the AI train could be hea...,0.001767,0.095859,0.902373
398,I broke Chat GPT #OpenAIChat #ChatGPT https://...,0.208422,0.690335,0.101243


In [ ]:
analysis_df

,tweet,positive_prob,neutral_prob,negative_prob
0,#ChatGPT thank u for my pitch deck bullets.,0.004381,0.154541,0.841077
1,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,0.024142,0.435439,0.540419
2,"Excellent piece.""Much of what is taught in #ST...",0.008182,0.176340,0.815478
3,"#ChatGPT gets it… solve cons from list A, with...",0.030797,0.497710,0.471493
4,“If you’re doing original reporting and uneart...,0.143114,0.592953,0.263933
...,...,...,...,...
995,OpenAI's #ChatGPT is writing a new book.\n\n👇 ...,0.006266,0.547368,0.446367
996,I am getting more impressed by #ChatGPT’s abil...,0.002790,0.048222,0.948988
997,#ChatGPT is better than human authors.\n\n👇 Click,0.039165,0.303260,0.657574
998,Google announced a new ai chatGPT application....,0.001416,0.019285,0.979299
